In [23]:
import pandas as pd
import os
import re

In [6]:
# Define file directories
MOVIELENS_DIR = '../Raw_Data/'
USER_DATA_FILE = 'users.dat'
MOVIE_DATA_FILE = 'movies.dat'
RATING_DATA_FILE = 'ratings.dat'

### `MOVIE DATA`


**Input:**  
Raw movie information containing the following fields:  
- `movie_id`
- `title` (including year: Ex: Toy story (1995))
- `genres` (e.g., multiple genres separated by delimiters)

**Output:**  
The raw data is cleaned and normalized into three separate relational tables to support efficient querying and database normalization:

1. **`movie`**  
   - Fields: `movie_id`, `name`, `year` 
   - Stores basic movie metadata.

2. **`genre`**  
   - Fields: `genre_id`, `genre`  
   - Contains the unique list of movie genres.

3. **`movie_genre`**  
   - Fields: `movie_id`, `genre_id`  
   - Maps each movie to its corresponding genres (many-to-many relationship).

In [36]:
movies = pd.read_csv(os.path.join(MOVIELENS_DIR, MOVIE_DATA_FILE), 
                    sep='::', 
                    engine='python', 
                    encoding='latin-1',
                    names=['movie_id', 'title', 'genres'])
movies['movie_id'] = movies['movie_id'] - 1
movies.head(5)

,movie_id,title,genres
0,0,Toy Story (1995),Animation|Children's|Comedy
1,1,Jumanji (1995),Adventure|Children's|Fantasy
2,2,Grumpier Old Men (1995),Comedy|Romance
3,3,Waiting to Exhale (1995),Comedy|Drama
4,4,Father of the Bride Part II (1995),Comedy


In [ ]:
# Extract name and year from title
def extract_title_year(title):
    match = re.match(r'^(.*)\s\((\d{4})\)$', title)
    if match:
        return match.group(1), int(match.group(2))
    return title, None

movies[['name', 'year']] = movies['title'].apply(
    lambda x: pd.Series(extract_title_year(x))
)
movies['year'] = movies['year'].astype('Int64')
# Drop the original title column
movies.drop(columns=['title'], inplace=True)

# Display updated movie table
print("Updated Movie Table:")
movies.head(5)

Updated Movie Table:


,movie_id,genres,name,year
0,0,Animation|Children's|Comedy,Toy Story,1995
1,1,Adventure|Children's|Fantasy,Jumanji,1995
2,2,Comedy|Romance,Grumpier Old Men,1995
3,3,Comedy|Drama,Waiting to Exhale,1995
4,4,Comedy,Father of the Bride Part II,1995


In [40]:
# Split genres into separate rows
movie_genre_pairs = []
genre_set = set()

for _, row in movies.iterrows():
    genres = row['genres'].split('|')
    for genre in genres:
        genre_set.add(genre)
        movie_genre_pairs.append({'movie_id': row['movie_id'], 'genre': genre})

# Create genre table with unique IDs
genre_list = sorted(list(genre_set))
genre_df = pd.DataFrame({
    'genre_id': range(0, len(genre_list)),
    'genre': genre_list
})

In [41]:
# Create movie_genre mapping with genre_id
movie_genre_df = pd.DataFrame(movie_genre_pairs)
movie_genre_df = movie_genre_df.merge(genre_df, on='genre')[['movie_id', 'genre_id']]

# Create movie table
movie_df = movies[['movie_id', 'name', "year"]]

In [44]:
movie_df.to_csv("../Preprocessed_Data/movies.csv", index=False)
genre_df.to_csv("../Preprocessed_Data/genres.csv", index=False)
movie_genre_df.to_csv("../Preprocessed_Data/movies-genre.csv", index=False)